In [41]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly

In [42]:
# Importação dos daddos
df_selic_serie_historica=pd.read_csv('https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=csv', sep=';')
df_selic_serie_historica.to_csv('./output/selic_serie_historica_df.csv')


In [43]:
# Parse de tratamento dos dados
df_selic=pd.read_csv('./output/selic_serie_historica_df.csv')
df_selic=df_selic.drop(columns=['Unnamed: 0'])
df_selic['data']=df_selic.apply(lambda x: datetime.strptime(x.data,'%d/%m/%Y').date(),axis=1)
df_selic=df_selic.set_index('data')
df_selic['valor']=df_selic.apply(lambda x: float(x.valor.replace(',','.')),axis=1)
df_selic=df_selic.sort_index(ascending=False)

In [44]:
def converte_ad_aa(taxa_ad):
    # recebi taxa ao dia em porcentagem
    taxa_ad=taxa_ad/100                 #tira porcentagem e coloca em fração
    #juros sobre juros e considera 252 dias
    return round(((1+taxa_ad)**(252)-1)*100,2)

In [45]:
df_selic['selic_aa']=df_selic.apply(lambda x: converte_ad_aa(x.valor),axis=1)

In [47]:
# Definição de número índice adotando 100 em 2000-01-01
# def num_indice_selic():
#     if 

for i in range(0,len(df_selic)):
    if i==0:
        #último valor
        df_selic.loc[df_selic.index[-1-i],'num_indice']=1
    else:
        df_selic.loc[df_selic.index[-1-i],'num_indice']=df_selic.loc[df_selic.index[-1-i+1],'num_indice']*(1+df_selic.loc[df_selic.index[-1-i+1],'valor']/100)


In [50]:

# Atribuindo o número índice de 100 para 2000-01-03
df_selic['num_indice']=df_selic.apply(lambda x: x.num_indice/df_selic.loc[datetime.strptime('2000-01-03','%Y-%m-%d').date(),'num_indice']*100,axis=1)
# df_selic['num_indice']=
df_selic.head(36)

,valor,selic_aa,num_indice
data,,,
2022-08-30,0.050788,13.65,1316.561489
2022-08-29,0.050788,13.65,1315.893173
2022-08-26,0.050788,13.65,1315.225196
2022-08-25,0.050788,13.65,1314.557559
2022-08-24,0.050788,13.65,1313.890260
2022-08-23,0.050788,13.65,1313.223300
2022-08-22,0.050788,13.65,1312.556679
2022-08-19,0.050788,13.65,1311.890396
2022-08-18,0.050788,13.65,1311.224451


In [51]:
df_selic.iloc[0,-1]/df_selic.iloc[1,-1]

1.00050788

In [60]:
df_selic.loc[datetime.strptime('2022-08-29','%Y-%m-%d').date(),'num_indice']/df_selic.loc[datetime.strptime('2021-08-30','%Y-%m-%d').date(),'num_indice']

1.1011066754250047

In [52]:
df_selic.to_csv('./output/selic_serie_historica_ad_aa.csv')

In [61]:
# Gráfico iterativo utilizando o plotly
#a partir de 2000 - Juros na ordem de grandeza atual, para não destorcer o gráfico
datas=df_selic.index[df_selic.index>datetime.strptime('2000-01-01','%Y-%m-%d').date()]
fig=px.line(x=datas,y=df_selic.loc[datas,'selic_aa'])
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/selic_serie_historica_grafico.html')
# fig.show()

'./output/selic_serie_historica_grafico.html'

In [62]:
valor_periodo=df_selic.loc[datetime.strptime('2022-08-03','%Y-%m-%d').date():datetime.strptime('2022-06-17','%Y-%m-%d').date(),'valor'].sum()
valor_periodo
# 17/06/2022 - 03/08/2022

1.667258

In [63]:

dias=len(df_selic.loc[datetime.strptime('2022-08-03','%Y-%m-%d').date():datetime.strptime('2022-06-17','%Y-%m-%d').date(),'valor'])